## **GETTING TO KNOW OUR DATA**

In [1]:
!wget -qnc -P ./movielens https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/movielens/users.dat
!wget -qnc -P ./movielens https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/movielens/ratings.dat
!wget -qnc -P ./movielens https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/movielens/movies.dat

'wget' is not recognized as an internal or external command,
operable program or batch file.


'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
unames = ['user_id', 'gender', 'age', 'occupation', "zip"]
users = pd.read_table('https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/movielens/users.dat', sep="::", header=None, names=unames,
                      engine='python')
print(users.shape)

(6040, 5)


In [3]:
users

,user_id,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [4]:
rnames = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_table("https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/movielens/ratings.dat", sep="::", header=None,
                      names=rnames, engine="python")
print(ratings.shape)

(1000209, 4)


In [5]:
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [6]:
mnames = ['movie_id', "title", "genres"]
movies = pd.read_table("https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/movielens/movies.dat", sep="::", header=None,
                      names=mnames, engine="python")
print(movies.shape)

(3883, 3)


In [7]:
display("users", "ratings", "movies.head()")

'users'

'ratings'

'movies.head()'

In [8]:
ratings.user_id.unique()

array([   1,    2,    3, ..., 6038, 6039, 6040], dtype=int64)

In [ ]:
len(ratings['user_id'].unique())

6040

In [ ]:
len(users.user_id.unique())

6040

In [ ]:
age_categories = {
    "Children": (1, 11),
    "PreTeen/Teen": (11, 20),
    "Young Adults": (20, 30),
    "Adults-1": (30, 40),
    "Adults-2": (40,50),
    "Middle Aged Adults": (50,60),
    "Senior Adults": (60, float('inf'))
}

def categorize_age(age):
    for category, (min_age, max_age) in age_categories.items():
        if min_age <= age < max_age:
            return category
    print(min_age, age, max_age)
    return "Unknown"

In [ ]:
users['Category'] = users['age'].apply(categorize_age)

In [ ]:
users

,user_id,gender,age,occupation,zip,Category
0,1,F,1,10,48067,Children
1,2,M,56,16,70072,Middle Aged Adults
2,3,M,25,15,55117,Young Adults
3,4,M,45,7,02460,Adults-2
4,5,M,25,20,55455,Young Adults
...,...,...,...,...,...,...
6035,6036,F,25,15,32603,Young Adults
6036,6037,F,45,1,76006,Adults-2
6037,6038,F,56,1,14706,Middle Aged Adults
6038,6039,F,45,0,01060,Adults-2


In [ ]:
user_ratings = pd.merge(ratings, users)
display("users", "ratings", "user_ratings.head()")

'users'

'ratings'

'user_ratings.head()'

In [ ]:
data = pd.merge(user_ratings, movies)
display("user_ratings", "movies", "data.head()")

'user_ratings'

'movies'

'data.head()'

## **CONTENT BASED FILTERING**

In [ ]:
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [ ]:
features = movies['genres']
features

0        Animation|Children's|Comedy
1       Adventure|Children's|Fantasy
2                     Comedy|Romance
3                       Comedy|Drama
4                             Comedy
                    ...             
3878                          Comedy
3879                           Drama
3880                           Drama
3881                           Drama
3882                  Drama|Thriller
Name: genres, Length: 3883, dtype: object

In [ ]:
#Converting text to Feature Vectors
vectorizer = TfidfVectorizer()
#use Scikit-Learn tfor implimentation of Vectorization

In [ ]:
feature_vectors = vectorizer.fit_transform(features)

In [ ]:
print(feature_vectors)

  (0, 4)	0.34435071658181016
  (0, 3)	0.5917143302654576
  (0, 2)	0.7289010463348881
  (1, 8)	0.6953995417662312
  (1, 1)	0.49981350223804
  (1, 3)	0.5163389780869351
  (2, 15)	0.8194348169667283
  (2, 4)	0.5731723830949155
  (3, 7)	0.655026247791881
  (3, 4)	0.7556061240512079
  (4, 4)	1.0
  (5, 17)	0.5261446495072489
  (5, 5)	0.6710563133242528
  (5, 0)	0.5223554653131872
  (6, 15)	0.8194348169667283
  (6, 4)	0.5731723830949155
  (7, 1)	0.6955152827862694
  (7, 3)	0.7185113022150282
  (8, 0)	1.0
  (9, 17)	0.544085722052581
  (9, 0)	0.5401673299903467
  (9, 1)	0.6420202354051012
  (10, 7)	0.4449744399512637
  (10, 15)	0.7338393352398338
  (10, 4)	0.5133006700217896
  :	:
  (3866, 7)	0.5238469433608234
  (3867, 9)	0.6076390878136414
  (3867, 16)	0.6076390878136414
  (3867, 17)	0.511419082477386
  (3868, 11)	1.0
  (3869, 11)	1.0
  (3870, 11)	1.0
  (3871, 11)	1.0
  (3872, 11)	1.0
  (3873, 4)	1.0
  (3874, 7)	0.655026247791881
  (3874, 4)	0.7556061240512079
  (3875, 1)	0.5208144676513746
 

In [ ]:
similarity = cosine_similarity(feature_vectors)
print(similarity)
print(similarity.shape) # this has calculated similarity of each with the other one

[[1.         0.30552517 0.19737232 ... 0.         0.         0.        ]
 [0.30552517 1.         0.         ... 0.         0.         0.        ]
 [0.19737232 0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         1.         0.52384694]
 [0.         0.         0.         ... 1.         1.         0.52384694]
 [0.         0.         0.         ... 0.52384694 0.52384694 1.        ]]
(3883, 3883)


In [ ]:
movie_name = input()

shawshank redepmtion


In [ ]:
list_titles = movies['title'].tolist()
#find the closest matches to it
closest_match = difflib.get_close_matches(movie_name,list_titles)[0]
print(closest_match)

Shawshank Redemption, The (1994)


In [ ]:
# finding the index of the movie with title

index_movie = movies[movies.title == closest_match]['movie_id'].values[0]
print(index_movie)

318


In [ ]:
similarity_score = list(enumerate(similarity[index_movie]))
# print(similarity_score)

# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(13, 1.0), (25, 1.0), (26, 1.0), (29, 1.0), (30, 1.0), (35, 1.0), (39, 1.0), (42, 1.0), (52, 1.0), (54, 1.0), (56, 1.0), (61, 1.0), (66, 1.0), (77, 1.0), (79, 1.0), (81, 1.0), (82, 1.0), (89, 1.0), (92, 1.0), (95, 1.0), (112, 1.0), (118, 1.0), (119, 1.0), (122, 1.0), (125, 1.0), (128, 1.0), (136, 1.0), (140, 1.0), (141, 1.0), (145, 1.0), (146, 1.0), (148, 1.0), (152, 1.0), (157, 1.0), (165, 1.0), (173, 1.0), (182, 1.0), (189, 1.0), (191, 1.0), (192, 1.0), (195, 1.0), (199, 1.0), (207, 1.0), (209, 1.0), (211, 1.0), (212, 1.0), (217, 1.0), (230, 1.0), (251, 1.0), (258, 1.0), (260, 1.0), (261, 1.0), (265, 1.0), (266, 1.0), (268, 1.0), (269, 1.0), (274, 1.0), (276, 1.0), (278, 1.0), (279, 1.0), (294, 1.0), (296, 1.0), (297, 1.0), (303, 1.0), (304, 1.0), (305, 1.0), (306, 1.0), (311, 1.0), (315, 1.0), (318, 1.0), (322, 1.0), (327, 1.0), (330, 1.0), (333, 1.0), (337, 1.0), (343, 1.0), (350, 1.0), (354, 1.0), (361, 1.0), (365, 1.0), (371, 1.0), (377, 1.0), (381, 1.0), (382, 1.0), (384, 1.0),

In [ ]:
print('Suggested Movies')
i=1
content=[]
for movie in sorted_similar_movies[:20]:
  index = movie[0]
  title = movies[movies.index == index]['title'].values[0]
  print(title)
  content.append(title)

Suggested Movies
Nixon (1995)
Othello (1995)
Now and Then (1995)
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
Dangerous Minds (1995)
Dead Man Walking (1995)
Cry, the Beloved Country (1995)
Restoration (1995)
Lamerica (1994)
Georgia (1995)
Home for the Holidays (1995)
Mr. Holland's Opus (1995)
Two Bits (1995)
Crossing Guard, The (1995)
White Balloon, The (Badkonake Sefid ) (1995)
Antonia's Line (Antonia) (1995)
Once Upon a Time... When We Were Colored (1995)
Journey of August King, The (1995)
Beautiful Girls (1996)
Hate (Haine, La) (1995)


## **POPULARITY BASED FILTERING**

In [ ]:
ratings_by_title = data.groupby('title').count()['rating'].reset_index()
ratings_by_title.rename(columns={'rating':'num_ratings'},inplace=True)
ratings_by_title.head()

,title,num_ratings
0,"$1,000,000 Duck (1971)",37
1,'Night Mother (1986),70
2,'Til There Was You (1997),52
3,"'burbs, The (1989)",303
4,...And Justice for All (1979),199


In [ ]:
avg_rating = data.groupby('title').mean()['rating'].reset_index()
avg_rating.rename(columns={'rating':'avg_rating'},inplace=True)
avg_rating

TypeError: Could not convert MFMMFFFMMFMMMMMFMMFMFFMMMFMFMFMFMFFFM to numeric

In [ ]:
avg_rating = data.groupby('title')#.mean()['rating'].reset_index()
avg_rating_mean = avg_rating['rating'].mean().reset_index()
avg_rating=ratings_by_title.merge(avg_rating_mean,on='title')
avg_rating.rename(columns={'rating':'avg_rating'},inplace=True)
avg_rating

,title,num_ratings,avg_rating
0,"$1,000,000 Duck (1971)",37,3.027027
1,'Night Mother (1986),70,3.371429
2,'Til There Was You (1997),52,2.692308
3,"'burbs, The (1989)",303,2.910891
4,...And Justice for All (1979),199,3.713568
...,...,...,...
3701,"Zed & Two Noughts, A (1985)",29,3.413793
3702,Zero Effect (1998),301,3.750831
3703,Zero Kelvin (Kjærlighetens kjøtere) (1995),2,3.500000
3704,Zeus and Roxanne (1997),23,2.521739


In [ ]:
popular_movies = ratings_by_title.merge(avg_rating,on='title')
popular_movies

,title,num_ratings_x,num_ratings_y,avg_rating
0,"$1,000,000 Duck (1971)",37,37,3.027027
1,'Night Mother (1986),70,70,3.371429
2,'Til There Was You (1997),52,52,2.692308
3,"'burbs, The (1989)",303,303,2.910891
4,...And Justice for All (1979),199,199,3.713568
...,...,...,...,...
3701,"Zed & Two Noughts, A (1985)",29,29,3.413793
3702,Zero Effect (1998),301,301,3.750831
3703,Zero Kelvin (Kjærlighetens kjøtere) (1995),2,2,3.500000
3704,Zeus and Roxanne (1997),23,23,2.521739


In [ ]:
popular_movies = avg_rating#ratings_by_title.merge(avg_rating,on='title')
popular_movies

,title,num_ratings,avg_rating
0,"$1,000,000 Duck (1971)",37,3.027027
1,'Night Mother (1986),70,3.371429
2,'Til There Was You (1997),52,2.692308
3,"'burbs, The (1989)",303,2.910891
4,...And Justice for All (1979),199,3.713568
...,...,...,...
3701,"Zed & Two Noughts, A (1985)",29,3.413793
3702,Zero Effect (1998),301,3.750831
3703,Zero Kelvin (Kjærlighetens kjøtere) (1995),2,3.500000
3704,Zeus and Roxanne (1997),23,2.521739


In [ ]:
popular_movies = popular_movies[popular_movies['num_ratings']>=400].sort_values('avg_rating',ascending=False)

In [ ]:
np.shape(popular_movies)
popular_movies.head()

,title,num_ratings,avg_rating
2940,Seven Samurai (The Magnificent Seven) (Shichin...,628,4.560510
2970,"Shawshank Redemption, The (1994)",2227,4.554558
1354,"Godfather, The (1972)",2223,4.524966
713,"Close Shave, A (1995)",657,4.520548
3504,"Usual Suspects, The (1995)",1783,4.517106


In [ ]:
print(popular_movies['title'].head(20))
#Ending the Popularity based filtering

2940    Seven Samurai (The Magnificent Seven) (Shichin...
2970                     Shawshank Redemption, The (1994)
1354                                Godfather, The (1972)
713                                 Close Shave, A (1995)
3504                           Usual Suspects, The (1995)
2901                              Schindler's List (1993)
3675                           Wrong Trousers, The (1993)
3218        Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
2711                       Raiders of the Lost Ark (1981)
2738                                   Rear Window (1954)
3153            Star Wars: Episode IV - A New Hope (1977)
3325                                Third Man, The (1949)
975     Dr. Strangelove or: How I Learned to Stop Worr...
3551    Wallace & Gromit: The Best of Aardman Animatio...
3381                         To Kill a Mockingbird (1962)
965                               Double Indemnity (1944)
609                                     Casablanca (1942)
3015          

# **COLLABORATIVE FILTERING**

In [ ]:
x = data.groupby('user_id').count()['rating'] > 75
critic_users = x[x].index
#created users that behae like critics and passed a threshold of 75 total ratings


In [ ]:
critic_users

Index([   2,    5,    8,    9,   10,   11,   13,   15,   17,   18,
       ...
       6024, 6025, 6026, 6030, 6032, 6035, 6036, 6037, 6039, 6040],
      dtype='int64', name='user_id', length=3462)

In [ ]:
filtered_rating = data[data['user_id'].isin(critic_users)]
filtered_rating

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,Category,title,genres
1,2,1193,5,978298413,M,56,16,70072,Middle Aged Adults,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,Young Adults,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,Middle Aged Adults,One Flew Over the Cuckoo's Nest (1975),Drama
5,18,1193,4,978156168,F,18,3,95825,PreTeen/Teen,One Flew Over the Cuckoo's Nest (1975),Drama
6,19,1193,5,982730936,M,1,10,48073,Children,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,M,18,17,47901,PreTeen/Teen,Modulations (1998),Documentary
1000205,5675,2703,3,976029116,M,35,14,30030,Adults-1,Broken Vessels (1998),Drama
1000206,5780,2845,1,958153068,M,18,17,92886,PreTeen/Teen,White Boys (1999),Drama
1000207,5851,3607,5,957756608,F,18,20,55410,PreTeen/Teen,One Little Indian (1973),Comedy|Drama|Western


In [ ]:
filtered_popular_rating = filtered_rating[filtered_rating['title'].isin(popular_movies['title'])]
filtered_popular_rating

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,Category,title,genres
1,2,1193,5,978298413,M,56,16,70072,Middle Aged Adults,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,Young Adults,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,Middle Aged Adults,One Flew Over the Cuckoo's Nest (1975),Drama
5,18,1193,4,978156168,F,18,3,95825,PreTeen/Teen,One Flew Over the Cuckoo's Nest (1975),Drama
6,19,1193,5,982730936,M,1,10,48073,Children,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...,...,...,...,...,...
943424,5831,3791,3,1024074425,M,25,1,92120,Young Adults,Footloose (1984),Drama
943425,5840,3791,4,962652226,F,1,0,44260,Children,Footloose (1984),Drama
943426,5880,3791,4,962219650,M,25,1,08904,Young Adults,Footloose (1984),Drama
943427,5956,3791,2,1001192546,F,18,4,02142,PreTeen/Teen,Footloose (1984),Drama


In [ ]:
user_gender = input()

M


In [ ]:
user_age = int(input())

15


In [ ]:
user_category = categorize_age(user_age)
user_category

'PreTeen/Teen'

In [ ]:
fr_user_gender = filtered_popular_rating.loc[filtered_rating['gender']== user_gender]
fr_user_gender.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,Category,title,genres
1,2,1193,5,978298413,M,56,16,70072,Middle Aged Adults,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,Young Adults,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,Middle Aged Adults,One Flew Over the Cuckoo's Nest (1975),Drama
6,19,1193,5,982730936,M,1,10,48073,Children,One Flew Over the Cuckoo's Nest (1975),Drama
9,33,1193,5,978557765,M,45,3,55421,Adults-2,One Flew Over the Cuckoo's Nest (1975),Drama


In [ ]:
category_ratings = fr_user_gender.pivot_table(values="rating", index="title", columns="Category", aggfunc="mean")
category_ratings.head()

Category,Adults-1,Adults-2,Children,Middle Aged Adults,PreTeen/Teen,Young Adults
title,,,,,,
10 Things I Hate About You (1999),2.976744,2.937500,3.428571,3.866667,3.400000,3.280822
101 Dalmatians (1961),3.705882,3.827586,3.200000,3.344828,3.202899,3.539683
12 Angry Men (1957),4.397727,4.230769,5.000000,4.338235,4.237288,4.376712
"13th Warrior, The (1999)",3.260274,3.259259,3.214286,3.472222,3.023077,3.159292
"20,000 Leagues Under the Sea (1954)",3.734694,3.653846,2.714286,3.837500,3.515625,3.688525


In [ ]:
user_category_ratings = category_ratings.sort_values(by=user_category, ascending=False)
user_category_ratings

Category,Adults-1,Adults-2,Children,Middle Aged Adults,PreTeen/Teen,Young Adults
title,,,,,,
"Shawshank Redemption, The (1994)",4.524793,4.458333,4.566667,4.279412,4.715827,4.565455
"Usual Suspects, The (1995)",4.314721,4.169231,4.363636,4.340206,4.676923,4.568224
Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954),4.567308,4.543478,4.416667,4.687500,4.625000,4.556250
"Godfather, The (1972)",4.625000,4.474138,4.741935,4.651613,4.577586,4.629423
Star Wars: Episode V - The Empire Strikes Back (1980),4.177966,4.125000,4.285714,4.115183,4.558904,4.476005
...,...,...,...,...,...,...
"Haunting, The (1999)",2.500000,2.571429,2.062500,2.266667,2.322917,2.216667
Congo (1995),2.240506,2.500000,2.250000,2.750000,2.176471,2.056497
Wild Wild West (1999),2.097345,2.119048,2.047619,2.368421,2.090909,2.065637


In [ ]:
#Movies recommended for user based on collaborative filtering:
sorted_pivot_table = user_category_ratings.count(axis=1).sort_values(ascending=False)
final_movies = sorted_pivot_table.head(15)

In [ ]:
final_movies

title
Shawshank Redemption, The (1994)          6
Analyze This (1999)                       6
American Werewolf in London, An (1981)    6
Road Trip (2000)                          6
Grumpy Old Men (1993)                     6
Antz (1998)                               6
Crying Game, The (1992)                   6
Nightmare on Elm Street, A (1984)         6
Midsummer Night's Dream, A (1999)         6
King Kong (1933)                          6
Sleeping Beauty (1959)                    6
Thin Red Line, The (1998)                 6
F/X (1986)                                6
Singles (1992)                            6
Clueless (1995)                           6
dtype: int64

**TRY HYBRID**

In [ ]:
movie_name = input()

shawshank redemption


In [ ]:
list_titles = movies['title'].tolist()
#find the closest matches to it
closest_match = difflib.get_close_matches(movie_name,list_titles)[0]
print(closest_match)

Shawshank Redemption, The (1994)


In [ ]:
import itertools as it

# finding the index of the movie with title

index_movie = movies[movies.title == closest_match]['movie_id'].values[0]
print('Suggested Movies')
i=1
content_list=[]
for movie in sorted_similar_movies[:20]:
  index = movie[0]
  title = movies[movies.index == index]['title'].values[0]
  content_list.append(title)
conW=int(0.4*len(content_list))
content={}
for x in content_list:
  content[x]="con"
content=dict(it.islice(content.items(),conW))
content

Suggested Movies


{'Nixon (1995)': 'con',
 'Othello (1995)': 'con',
 'Now and Then (1995)': 'con',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)': 'con',
 'Dangerous Minds (1995)': 'con',
 'Dead Man Walking (1995)': 'con',
 'Cry, the Beloved Country (1995)': 'con',
 'Restoration (1995)': 'con'}

In [ ]:
popular_list=list(popular_movies['title'][:20])
popW=int(0.2*len(popular_list))
popular={}
for x in popular_list:
  popular[x]="pop"
popular=dict(it.islice(popular.items(),popW))
popular

{'Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)': 'pop',
 'Shawshank Redemption, The (1994)': 'pop',
 'Godfather, The (1972)': 'pop',
 'Close Shave, A (1995)': 'pop'}

In [ ]:
user_gender = input()
user_age = int(input())

M
15


In [ ]:
user_category = categorize_age(user_age)

fr_user_gender = filtered_popular_rating.loc[filtered_rating['gender']== user_gender]
category_ratings = fr_user_gender.pivot_table(values="rating", index="title", columns="Category", aggfunc="mean")
category_ratings

user_category_ratings = category_ratings.sort_values(by=user_category, ascending=False)
user_category_ratings
colab_list=list(user_category_ratings.index[:20])
# sorted_pivot_table = user_category_ratings.count(axis=1).sort_values(ascending=False)
# final_movies = sorted_pivot_table[:21]
colW=int(0.4*len(colab_list))
# tagging elements
colab={}
for x in colab_list:
  colab[x]="colab"
colab=dict(it.islice(colab.items(),colW))
colab

# final_movies

{'Shawshank Redemption, The (1994)': 'colab',
 'Usual Suspects, The (1995)': 'colab',
 'Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)': 'colab',
 'Godfather, The (1972)': 'colab',
 'Star Wars: Episode V - The Empire Strikes Back (1980)': 'colab',
 'Life Is Beautiful (La Vita è bella) (1997)': 'colab',
 'Crimes and Misdemeanors (1989)': 'colab',
 'Close Shave, A (1995)': 'colab'}

In [ ]:
vectorizer = TfidfVectorizer()
pop_v=vectorizer.fit_transform(popular)
cont_v=vectorizer.fit_transform(content)
colab_v=vectorizer.fit_transform(colab)
# print(pop_v, cont_v, colab_v)

In [ ]:
fm={}

# Update fm with values from dict1
for x, v in content.items():
  if x in fm:
    fm[x].append(v)
  else:
    fm[x] = [v]

# Update fm with values from dict2
for x, v in colab.items():
    if x in fm:
        fm[x].append(v)
    else:
        fm[x] = [v]

# Update fm with values from dict3
for x, v in popular.items():
    if x in fm:
        fm[x].append(v)
    else:
        fm[x] = [v]

print(fm)

{'Nixon (1995)': ['con'], 'Othello (1995)': ['con'], 'Now and Then (1995)': ['con'], 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)': ['con'], 'Dangerous Minds (1995)': ['con'], 'Dead Man Walking (1995)': ['con'], 'Cry, the Beloved Country (1995)': ['con'], 'Restoration (1995)': ['con'], 'Shawshank Redemption, The (1994)': ['colab', 'pop'], 'Usual Suspects, The (1995)': ['colab'], 'Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)': ['colab', 'pop'], 'Godfather, The (1972)': ['colab', 'pop'], 'Star Wars: Episode V - The Empire Strikes Back (1980)': ['colab'], 'Life Is Beautiful (La Vita è bella) (1997)': ['colab'], 'Crimes and Misdemeanors (1989)': ['colab'], 'Close Shave, A (1995)': ['colab', 'pop']}


In [ ]:
def contentFltr(conW, closest_match):
  index_movie = movies[movies.title == closest_match]['movie_id'].values[0]
  print('Suggested Movies')
  i=1
  content_list=[]
  for movie in sorted_similar_movies[:20]:
    index = movie[0]
    title = movies[movies.index == index]['title'].values[0]
    content_list.append(title)
  con=int(conW*len(content_list))
  content={}
  for x in content_list:
    content[x]="con"
  content=dict(it.islice(content.items(),con))
  return content

def colabFltr(colW):
  user_category = categorize_age(user_age)

  fr_user_gender = filtered_popular_rating.loc[filtered_rating['gender']== user_gender]
  category_ratings = fr_user_gender.pivot_table(values="rating", index="title", columns="Category", aggfunc="mean")
  category_ratings

  user_category_ratings = category_ratings.sort_values(by=user_category, ascending=False)
  user_category_ratings
  colab_list=list(user_category_ratings.index[:20])
  # sorted_pivot_table = user_category_ratings.count(axis=1).sort_values(ascending=False)
  # final_movies = sorted_pivot_table[:21]
  col=int(colW*len(colab_list))
  # tagging elements
  colab={}
  for x in colab_list:
    colab[x]="colab"
  colab=dict(it.islice(colab.items(),col))
  return colab

def popularFltr(popW):
  popular_list=list(popular_movies['title'][:20])
  pop=int(popW*len(popular_list))
  popular={}
  for x in popular_list:
    popular[x]="pop"
  popular=dict(it.islice(popular.items(),pop))
  return popular

def final_moviesFn(content, popular, colab):
  fm={}
  # Update fm with values from dict1
  for x, v in content.items():
    if x in fm:
      fm[x].append(v)
    else:
      fm[x]=[v]

  # Update fm with values from dict2
  for x, v in colab.items():
      if x in fm:
          fm[x].append(v)
      else:
          fm[x]=[v]

  # Update fm with values from dict3
  for x, v in popular.items():
      if x in fm:
          fm[x].append(v)
      else:
          fm[x]=[v]
  return fm

In [ ]:
conW, colW, popW=0.4,0.4,0.2

movie_name = input("Enter Movie Name: ")
list_titles = movies['title'].tolist()
#find the closest matches to it
closest_match = difflib.get_close_matches(movie_name,list_titles)[0]
print(closest_match)

watched=[]
watched.append(closest_match)

while(True):
  content=contentFltr(conW, closest_match)
  colab=colabFltr(colW)
  popular=popularFltr(popW)
  final_movies=final_moviesFn(content, popular, colab)
  if(newCon):
    for x in newCon:
      final_movies[x]=
  show_movies=list(final_movies.keys())
  newCon={}

  for x in watched:
    if x in show_movies:
      show_movies.remove(x)

  print(show_movies)

  movie_name = input("Enter Movie Name... type END to stop: ")
  if(movie_name=="END"):
    break
  list_titles = movies['title'].tolist()
  #find the closest matches to it
  closest_match = difflib.get_close_matches(movie_name,list_titles)[0]
  watched.append(closest_match)
  print(closest_match)

  if(closest_match in final_movies.keys()):
    filt=final_movies[closest_match]
    for x in filt:
      if(x=="con"):
        conW+=0.1
        print(x)
      elif(x=="pop"):
        popW+=0.1
        print(x)
      else:
        colW+=0.1
        print(x)
    else:
      newCon=contentFltr(conW/2, closest_match)

Enter Movie Name: shawshank redemption
Shawshank Redemption, The (1994)
Suggested Movies
['Nixon (1995)', 'Othello (1995)', 'Now and Then (1995)', 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 'Dangerous Minds (1995)', 'Dead Man Walking (1995)', 'Cry, the Beloved Country (1995)', 'Restoration (1995)', 'Usual Suspects, The (1995)', 'Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)', 'Godfather, The (1972)', 'Star Wars: Episode V - The Empire Strikes Back (1980)', 'Life Is Beautiful (La Vita è bella) (1997)', 'Crimes and Misdemeanors (1989)', 'Close Shave, A (1995)']
Enter Movie Name... type END to stop: Now and Then (1995)
Now and Then (1995)
con
Suggested Movies
['Nixon (1995)', 'Othello (1995)', 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 'Dangerous Minds (1995)', 'Dead Man Walking (1995)', 'Cry, the Beloved Country (1995)', 'Restoration (1995)', 'Lamerica (1994)', 'Georgia (1995)', 'Usual Suspects, The (1995)', 'Seven Samurai (The Magnificent 